## Routing

### Importations

In [1]:
from langchain_core.messages import HumanMessage, AIMessage
from pprint import pprint
from dotenv import load_dotenv
import os

### Ignore Warnings

In [2]:
%reload_ext dotenv
%dotenv

### Creating Tools

In [4]:
def addition(a: int, b:int) -> int:
    """Use this tool to add two integer numbers"""
    return a+b

### Binding Tools To Chat Model

In [5]:
from langchain_openai import ChatOpenAI

In [6]:
model = ChatOpenAI(temperature=0.0, model="gpt-4o-mini", api_key=os.getenv("OPENAI_API_KEY"))

In [7]:
model_with_tools = model.bind_tools([addition])

### Creating Graph

In [10]:
from langgraph.graph import StateGraph, START, END, MessageGraph, MessagesState
from langgraph.prebuilt import ToolNode, tools_condition


### Tool Node

In [11]:
def tool_calling_node(state: MessagesState):
    return [{"messages": model_with_tools.invoke(state["messages"])}]

### Building Graph

In [14]:
builder = StateGraph(MessagesState)

builder.add_node("tool_calling_node", tool_calling_node)
builder.add_node("tools", ToolNode([addition]))

builder.add_edge(START, "tool_calling_node")
builder.add_conditional_edges(
    "tool_calling_node",
    tools_condition
)

builder.add_edge("tools", END)
graph = builder.compile()